# Notebook 20B: Tuned Multi-Year Validation

**Same as Notebook 20, but with ALL tuning improvements applied.**

## Tuning Applied:
1. Higher initial uncertainty: 0.125 → 0.20
2. Minimum uncertainty floor: 0.06
3. Driver-specific minimums (VER: 0.05, PER: 0.10)
4. Adaptive learning rate (slower mid-season)
5. Measurement noise: +4%
6. Outlier detection (catastrophic failures)

## Expected Results:
- MAE: 0.135-0.145 (vs 0.151 baseline)
- Calibration: 60-70% (vs 15.8% baseline)
- Runtime: ~2 hours (5 seasons)

In [1]:
# CLEAN ERROR HANDLING
import logging
import sys
import warnings

warnings.filterwarnings('ignore')
logging.getLogger('fastf1').setLevel(logging.ERROR)
logging.getLogger('requests').setLevel(logging.ERROR)
logging.getLogger('urllib3').setLevel(logging.ERROR)
logging.getLogger('requests_cache').setLevel(logging.ERROR)
sys.tracebacklimit = 0

In [2]:
# Imports
import copy
import json
import time
from pathlib import Path

import fastf1 as ff1
import numpy as np
import pandas as pd

ff1.Cache.enable_cache('../data/raw/.fastf1_cache')

In [3]:
# TUNED PARAMETERS
TUNED_PARAMS = {
    'initial_uncertainty': 0.20,  # Was 0.125
    'min_uncertainty': 0.06,      # Was None
    'measurement_noise': 0.04,    # Was 0.0
    'driver_specific_mins': {
        'VER': 0.05,  # Consistent drivers
        'RUS': 0.05,
        'LEC': 0.06,
        'HAM': 0.06,
        'NOR': 0.06,
        'ALO': 0.06,
        'SAI': 0.07,
        'PER': 0.10,  # High-variance driver
        'default': 0.07
    }
}

SEASON_TRANSITIONS = [
    {'from': 2020, 'to': 2021, 'type': 'stable'},
    {'from': 2021, 'to': 2022, 'type': 'regulation_change'},
    {'from': 2022, 'to': 2023, 'type': 'stable'},
    {'from': 2023, 'to': 2024, 'type': 'stable'},
    {'from': 2024, 'to': 2025, 'type': 'stable'}
]

TRACKED_DRIVERS = ['VER', 'HAM', 'LEC', 'NOR', 'SAI', 'PER', 'RUS', 'ALO']

## Data Extraction Functions

In [4]:
def extract_race_results(year, race_name):
    """Extract results with clean error handling."""
    time.sleep(0.5)
    
    try:
        quali = ff1.get_session(year, race_name, 'Q')
        quali.load(laps=False, telemetry=False, weather=False)
        
        race = ff1.get_session(year, race_name, 'R')
        race.load(laps=False, telemetry=False, weather=False)
        
        results = {}
        
        for _, row in quali.results.iterrows():
            driver = row['Abbreviation']
            quali_pos = row['Position']
            
            if pd.notna(driver) and pd.notna(quali_pos):
                results[driver] = {'quali_pos': int(quali_pos)}
        
        for _, row in race.results.iterrows():
            driver = row['Abbreviation']
            race_pos = row['Position']
            
            dnf = row.dnf if hasattr(row, 'dnf') else False
            status = str(row['Status']) if 'Status' in row else ''
            if not dnf and status:
                dnf = 'Finished' not in status and '+' not in status
            
            if pd.notna(driver) and driver in results:
                if pd.notna(race_pos):
                    results[driver]['race_pos'] = int(race_pos)
                results[driver]['dnf'] = dnf
        
        return results
        
    except Exception as e:
        print(f"    ❌ Error: {type(e).__name__}")
        return None

In [5]:
def extract_season_characteristics(year):
    """Extract prior year characteristics."""
    schedule = ff1.get_event_schedule(year)
    driver_stats = {}
    
    for _, event in schedule.iterrows():
        if event['EventFormat'] != 'conventional':
            continue
        
        race_name = event['EventName']
        
        try:
            results = extract_race_results(year, race_name)
            if not results:
                continue
            
            for driver, data in results.items():
                if driver not in driver_stats:
                    driver_stats[driver] = {
                        'quali_positions': [],
                        'dnfs': 0,
                        'races': 0
                    }
                
                driver_stats[driver]['quali_positions'].append(data['quali_pos'])
                driver_stats[driver]['races'] += 1
                if data.get('dnf', False):
                    driver_stats[driver]['dnfs'] += 1
        
        except Exception:
            continue
    
    characteristics = {}
    for driver, stats in driver_stats.items():
        if stats['races'] > 0:
            avg_quali_pos = np.mean(stats['quali_positions'])
            avg_pace = 1.0 - (avg_quali_pos - 1) / 19
            dnf_rate = stats['dnfs'] / stats['races']
            
            characteristics[driver] = {
                'avg_quali_pace': float(avg_pace),
                'dnf_rate': float(dnf_rate),
                'races_completed': stats['races']
            }
    
    print(f"  ✅ Extracted characteristics for {len(characteristics)} drivers")
    return characteristics

## Tuned Bayesian Update Functions

In [6]:
def tuned_uncertainty_update(prior_uncertainty, observation_variance, n_observations, 
                             driver_code, error_magnitude):
    """
    Bayesian uncertainty update with tuning improvements.
    
    Improvements:
    1. Minimum uncertainty floor
    2. Driver-specific minimums
    3. Outlier detection (catastrophic failures)
    """
    # Standard Bayesian update
    prior_var = prior_uncertainty ** 2
    posterior_var = 1.0 / (1.0/prior_var + n_observations/observation_variance)
    posterior_uncertainty = np.sqrt(posterior_var)
    
    # TUNING: Outlier detection
    if error_magnitude > 0.5:
        posterior_uncertainty = posterior_uncertainty * 1.2
    
    # TUNING: Apply minimum floor
    driver_min = TUNED_PARAMS['driver_specific_mins'].get(
        driver_code, 
        TUNED_PARAMS['driver_specific_mins']['default']
    )
    
    return float(max(posterior_uncertainty, driver_min))


def tuned_observation_variance(pace_history):
    """
    Calculate observation variance with measurement noise.
    
    TUNING: Always add 4% measurement noise.
    """
    if len(pace_history) < 2:
        return 0.08 + TUNED_PARAMS['measurement_noise']
    
    recent_paces = pace_history[-5:]
    obs_var = np.var(recent_paces)
    total_var = obs_var + TUNED_PARAMS['measurement_noise']
    
    min_var = 0.005
    return max(total_var, min_var)


def adaptive_learning_rate(races_seen, season_progress):
    """
    TUNING: Adaptive learning rate that slows mid-season.
    """
    base_alpha = max(0.05, 1.0 / (races_seen + 2))
    
    # Slow down during upgrade window
    if 0.4 < season_progress < 0.8:
        base_alpha *= 0.7
    
    return base_alpha

In [7]:
def tuned_bayesian_update(priors, race_results, season_progress):
    """Bayesian update with ALL tuning improvements."""
    posteriors = copy.deepcopy(priors)
    posteriors['week'] = priors.get('week', 0) + 1
    posteriors['races_seen'] = priors.get('races_seen', 0) + 1
    
    grid_size = 20
    
    for driver, result in race_results.items():
        if driver not in posteriors['drivers']:
            continue
        
        driver_data = posteriors['drivers'][driver]
        races_seen = driver_data.get('races_seen', 0)
        
        # TUNING: Adaptive learning rate
        alpha = adaptive_learning_rate(races_seen, season_progress)
        
        # Update pace
        observed_pace = 1.0 - (result['quali_pos'] - 1) / (grid_size - 1)
        prior_pace = driver_data['pace']['quali_pace']
        new_pace = (1 - alpha) * prior_pace + alpha * observed_pace
        
        driver_data['pace']['quali_pace'] = float(new_pace)
        
        # Store history
        if 'pace_history' not in driver_data:
            driver_data['pace_history'] = []
        driver_data['pace_history'].append(float(observed_pace))
        
        # Calculate error for outlier detection
        error = abs(prior_pace - observed_pace)
        
        # Update uncertainty with ALL tuning
        prior_uncertainty = driver_data['pace']['uncertainty']
        dnf = result.get('dnf', False)
        
        if dnf:
            obs_var = 0.10
            n_obs = 0.5
        else:
            obs_var = tuned_observation_variance(driver_data['pace_history'])
            n_obs = 1.0
        
        new_uncertainty = tuned_uncertainty_update(
            prior_uncertainty, obs_var, n_obs, driver, error
        )
        
        driver_data['pace']['uncertainty'] = float(new_uncertainty)
        driver_data['races_seen'] = races_seen + 1
    
    return posteriors

## Validation Functions

In [8]:
def calculate_mae(predictions, actuals):
    """Calculate mean absolute error."""
    errors = []
    for driver in predictions:
        if driver in actuals:
            error = abs(predictions[driver] - actuals[driver])
            errors.append(error)
    return np.mean(errors) if errors else 0.0


def calculate_calibration(predictions, uncertainties, actuals):
    """Calculate what % of predictions fall within 1σ."""
    within_1sigma = []
    
    for driver in predictions:
        if driver in actuals:
            pred = predictions[driver]
            unc = uncertainties[driver]
            actual = actuals[driver]
            error = abs(pred - actual)
            within_1sigma.append(error <= unc)
    
    return (sum(within_1sigma) / len(within_1sigma) * 100) if within_1sigma else 0.0

## Main Validation Loop

In [9]:
print("="*70)
print("TUNED MULTI-YEAR VALIDATION")
print("="*70)
print("\nTuning Applied:")
print(f"  Initial uncertainty: {TUNED_PARAMS['initial_uncertainty']}")
print(f"  Min uncertainty: {TUNED_PARAMS['min_uncertainty']}")
print(f"  Measurement noise: {TUNED_PARAMS['measurement_noise']}")
print("  Adaptive learning: Yes")
print("  Driver-specific mins: Yes")

all_validation_results = {
    'transitions': {},
    'summary': {
        'regulation_change': {'transitions': [], 'maes': [], 'calibrations': []},
        'stable': {'transitions': [], 'maes': [], 'calibrations': []}
    },
    'tuning_params': TUNED_PARAMS
}

TUNED MULTI-YEAR VALIDATION

Tuning Applied:
  Initial uncertainty: 0.2
  Min uncertainty: 0.06
  Measurement noise: 0.04
  Adaptive learning: Yes
  Driver-specific mins: Yes


In [10]:
for transition in SEASON_TRANSITIONS:
    from_year = transition['from']
    to_year = transition['to']
    trans_type = transition['type']
    trans_key = f"{from_year}_{to_year}"
    
    print(f"\n{'='*70}")
    print(f"TRANSITION: {from_year} → {to_year} ({trans_type})")
    print("="*70)
    
    # Extract prior year characteristics
    print(f"\n  Extracting {from_year} season characteristics...")
    prior_characteristics = extract_season_characteristics(from_year)
    
    # Initialize priors with TUNED parameters
    priors = {'week': 0, 'season': to_year, 'races_seen': 0, 'drivers': {}}
    
    # Set initial uncertainty
    if trans_type == 'regulation_change':
        initial_unc = 0.25
        print("  🔥 Regulation change detected! Increasing uncertainty...")
    else:
        initial_unc = TUNED_PARAMS['initial_uncertainty']
    
    for driver_code, char in prior_characteristics.items():
        priors['drivers'][driver_code] = {
            'pace': {
                'quali_pace': char['avg_quali_pace'],
                'uncertainty': initial_unc,
                'confidence': 'low'
            },
            'dnf_risk': {'rate': char['dnf_rate']},
            'races_seen': 0,
            'pace_history': []
        }
    
    # Validate across to_year season
    schedule = ff1.get_event_schedule(to_year)
    current_priors = copy.deepcopy(priors)
    
    transition_results = {
        'from_year': from_year,
        'to_year': to_year,
        'transition_type': trans_type,
        'drivers': {driver: {'errors': [], 'uncertainties': [], 'within_1sigma': [], 'within_2sigma': []} 
                   for driver in TRACKED_DRIVERS}
    }
    
    week = 0
    total_races = len([e for _, e in schedule.iterrows() if e['EventFormat'] == 'conventional'])
    
    for _, event in schedule.iterrows():
        if event['EventFormat'] != 'conventional':
            continue
        
        week += 1
        race_name = event['EventName']
        season_progress = week / total_races
        
        print(f"\n  Week {week}: {race_name}")
        
        # Extract actual results
        actual_results = extract_race_results(to_year, race_name)
        if not actual_results:
            continue
        
        print(f"    ✅ Extracted {len(actual_results)} drivers")
        
        # Make predictions and calculate metrics
        predictions = {}
        uncertainties = {}
        actuals = {}
        
        for driver in TRACKED_DRIVERS:
            if driver in current_priors['drivers'] and driver in actual_results:
                pred_pace = current_priors['drivers'][driver]['pace']['quali_pace']
                unc = current_priors['drivers'][driver]['pace']['uncertainty']
                actual_pace = 1.0 - (actual_results[driver]['quali_pos'] - 1) / 19
                
                predictions[driver] = pred_pace
                uncertainties[driver] = unc
                actuals[driver] = actual_pace
                
                error = abs(pred_pace - actual_pace)
                within_1 = error <= unc
                within_2 = error <= 2 * unc
                
                transition_results['drivers'][driver]['errors'].append(error)
                transition_results['drivers'][driver]['uncertainties'].append(unc)
                transition_results['drivers'][driver]['within_1sigma'].append(within_1)
                transition_results['drivers'][driver]['within_2sigma'].append(within_2)
        
        # Calculate metrics
        mae = calculate_mae(predictions, actuals)
        cal = calculate_calibration(predictions, uncertainties, actuals)
        
        print(f"    📊 MAE: {mae:.3f} | Cal(1σ): {cal:.1f}%")
        
        # Update priors with TUNED update
        current_priors = tuned_bayesian_update(current_priors, actual_results, season_progress)
    
    # Transition summary
    all_errors = []
    all_1sigma = []
    
    for driver in TRACKED_DRIVERS:
        all_errors.extend(transition_results['drivers'][driver]['errors'])
        all_1sigma.extend(transition_results['drivers'][driver]['within_1sigma'])
    
    trans_mae = np.mean(all_errors)
    trans_cal = sum(all_1sigma) / len(all_1sigma) * 100
    
    print(f"\n  {'='*70}")
    print(f"  TRANSITION SUMMARY: {from_year} → {to_year}")
    print(f"  {'='*70}")
    print(f"  MAE: {trans_mae:.3f}")
    print(f"  Calibration (1σ): {trans_cal:.1f}%")
    print(f"  Predictions: {len(all_errors)}")
    
    # Store results
    transition_results['summary_mae'] = trans_mae
    transition_results['summary_calibration'] = trans_cal
    transition_results['total_predictions'] = len(all_errors)
    
    all_validation_results['transitions'][trans_key] = transition_results
    all_validation_results['summary'][trans_type]['transitions'].append(trans_key)
    all_validation_results['summary'][trans_type]['maes'].append(trans_mae)
    all_validation_results['summary'][trans_type]['calibrations'].append(trans_cal)


TRANSITION: 2020 → 2021 (stable)

  Extracting 2020 season characteristics...


core           INFO 	Loading data for Austrian Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '44', '33', '4', '23', '11', '16', '55', '18', '3', '5', '10', '26', '31', '8', '20', '63', '99', '7', '6']
core           INFO 	Loading data for Austrian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '16', '4', '44', '55', '11', '10', '31', '99', '5', '6', '26', '23', '7', '63', '8', '20', '18', '3', '33']
core           INFO 	Loading data for Styrian Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req        

  ✅ Extracted characteristics for 23 drivers

  Week 1: Bahrain Grand Prix


core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '44', '77', '16', '10', '3', '4', '55', '14', '18', '11', '99', '22', '7', '63', '31', '6', '5', '47', '9']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '77', '4', '11', '16', '3', '55', '22', '18', '7', '99', '31', '63', '5', '47', '10', '6', '14', '9']


    ✅ Extracted 20 drivers
    📊 MAE: 0.093 | Cal(1σ): 71.4%

  Week 2: Emilia Romagna Grand Prix


...

    ✅ Extracted 20 drivers
    📊 MAE: 0.160 | Cal(1σ): 14.3%

  Week 17: Las Vegas Grand Prix


core           INFO 	Loading data for Las Vegas Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '55', '63', '81', '30', '14', '6', '16', '10', '27', '18', '31', '87', '43', '23', '12', '5', '22', '44']
core           INFO 	Loading data for Las Vegas Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '63', '12', '16', '55', '6', '27', '44', '31', '87', '14', '22', '10', '30', '43', '23', '5', '18', '4', '81']


    ✅ Extracted 20 drivers
    📊 MAE: 0.239 | Cal(1σ): 14.3%

  Week 18: Abu Dhabi Grand Prix


core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '63', '16', '14', '5', '31', '6', '22', '87', '55', '30', '12', '18', '44', '23', '27', '10', '43']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '81', '4', '16', '63', '14', '31', '44', '27', '18', '5', '87', '55', '22', '12', '23', '6', '30', '10', '43']


    ✅ Extracted 20 drivers
    📊 MAE: 0.137 | Cal(1σ): 42.9%

  TRANSITION SUMMARY: 2024 → 2025
  MAE: 0.144
  Calibration (1σ): 40.5%
  Predictions: 126


## Final Summary

In [11]:
print(f"\n{'='*70}")
print("TUNED VALIDATION COMPLETE!")
print("="*70)

all_errors = []
all_1sigma = []

for trans_key, data in all_validation_results['transitions'].items():
    for driver in TRACKED_DRIVERS:
        if driver in data['drivers']:
            all_errors.extend(data['drivers'][driver]['errors'])
            all_1sigma.extend(data['drivers'][driver]['within_1sigma'])

overall_mae = np.mean(all_errors)
overall_cal = sum(all_1sigma) / len(all_1sigma) * 100

print("\nOVERALL RESULTS:")
print(f"  Total Predictions: {len(all_errors)}")
print(f"  MAE: {overall_mae:.3f}")
print(f"  Calibration (1σ): {overall_cal:.1f}%")

print("\nBY TRANSITION TYPE:")
for trans_type in ['regulation_change', 'stable']:
    type_data = all_validation_results['summary'][trans_type]
    if type_data['maes']:
        print(f"\n{trans_type.upper()}:")
        print(f"  MAE: {np.mean(type_data['maes']):.3f}")
        print(f"  Calibration: {np.mean(type_data['calibrations']):.1f}%")

print(f"\n{'='*70}")
print("COMPARISON TO BASELINE:")
print("="*70)
print("Baseline (Notebook 20):")
print("  MAE: 0.151")
print("  Calibration: 15.8%")
print("\nTuned (Notebook 20B):")
print(f"  MAE: {overall_mae:.3f}")
print(f"  Calibration: {overall_cal:.1f}%")
print("\nImprovement:")
print(f"  MAE: {(0.151 - overall_mae) / 0.151 * 100:.1f}%")
print(f"  Calibration: {overall_cal / 15.8:.1f}x better")


TUNED VALIDATION COMPLETE!

OVERALL RESULTS:
  Total Predictions: 682
  MAE: 0.151
  Calibration (1σ): 43.8%

BY TRANSITION TYPE:

REGULATION_CHANGE:
  MAE: 0.121
  Calibration: 52.6%

STABLE:
  MAE: 0.159
  Calibration: 41.2%

COMPARISON TO BASELINE:
Baseline (Notebook 20):
  MAE: 0.151
  Calibration: 15.8%

Tuned (Notebook 20B):
  MAE: 0.151
  Calibration: 43.8%

Improvement:
  MAE: 0.1%
  Calibration: 2.8x better


## Save Results

In [12]:
output_path = Path('../data/processed/testing_files/validation')
output_path.mkdir(parents=True, exist_ok=True)

with open(output_path / 'tuned_multiyear_validation_results.json', 'w') as f:
    json.dump(all_validation_results, f, indent=2)

print(f"\n✅ Results saved to: {output_path / 'tuned_multiyear_validation_results.json'}")


✅ Results saved to: ../data/processed/testing_files/validation/tuned_multiyear_validation_results.json
